In [6]:
# Packages / libraries
import os #provides functions for interacting with the operating system
import numpy as np 
import pandas as pd
from datetime import datetime

In [7]:
global_confirmed =  pd.read_csv("covid_19_data/covid_19_time_series/time_series_covid19_confirmed_global.csv")
global_deaths =  pd.read_csv("covid_19_data/covid_19_time_series/time_series_covid19_deaths_global.csv")
global_recovered =  pd.read_csv("covid_19_data/covid_19_time_series/time_series_covid19_recovered_global.csv")

In [8]:
print (global_confirmed.shape)
print(global_deaths.shape)
print(global_recovered.shape)

global_confirmed.head()

(264, 95)
(264, 95)
(250, 95)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,607,665,714,784,840,906,933,996,1026,1092
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,446,467,475,494,518,539,548,562,584,609
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,1914,1983,2070,2160,2268,2418,2534,2629,2718,2811
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,638,646,659,673,673,696,704,713,717,717
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,19,19,19,19,19,19,24,24,24,24


In [9]:
global_confirmed2 = pd.melt(global_confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
global_deaths2 = pd.melt(global_deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
global_recovered2 = pd.melt(global_recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])

In [10]:
print("The Shape is: ", global_confirmed2.shape)
print("The Shape is: ", global_deaths2.shape)
print("The Shape is: ", global_recovered2.shape)


global_confirmed2.tail()

The Shape is:  (24024, 6)
The Shape is:  (24024, 6)
The Shape is:  (22750, 6)


,Province/State,Country/Region,Lat,Long,Date,value
24019,Saint Pierre and Miquelon,France,46.885200,-56.315900,4/21/20,1
24020,NaN,South Sudan,6.877000,31.307000,4/21/20,4
24021,NaN,Western Sahara,24.215500,-12.885800,4/21/20,6
24022,NaN,Sao Tome and Principe,0.186360,6.613081,4/21/20,4
24023,NaN,Yemen,15.552727,48.516388,4/21/20,1


In [11]:
#Converting the new column to dates

global_confirmed2['Date'] = pd.to_datetime(global_confirmed2['Date'])
global_deaths2['Date'] = pd.to_datetime(global_deaths2['Date'])
global_recovered2['Date'] = pd.to_datetime(global_recovered2['Date'])

In [12]:
# Renaming the Values
global_confirmed2.columns = global_confirmed2.columns.str.replace('value', 'Confirmed')
global_deaths2.columns = global_deaths2.columns.str.replace('value', 'Deaths')
global_recovered2.columns = global_recovered2.columns.str.replace('value', 'Recovered')

In [13]:
print(global_confirmed2.isnull().sum())
print(global_deaths2.isnull().sum())
print(global_recovered2.isnull().sum())

Province/State    16562
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
dtype: int64
Province/State    16562
Country/Region        0
Lat                   0
Long                  0
Date                  0
Deaths                0
dtype: int64
Province/State    16653
Country/Region        0
Lat                   0
Long                  0
Date                  0
Recovered             0
dtype: int64


In [14]:
# Dealing with NULL values

global_confirmed2['Province/State'].fillna(global_confirmed2['Country/Region'], inplace=True)
global_deaths2['Province/State'].fillna(global_deaths2['Country/Region'], inplace=True)
global_recovered2['Province/State'].fillna(global_recovered2['Country/Region'], inplace=True)

global_confirmed2.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
Confirmed         0
dtype: int64

In [15]:
#Combining all the three datasets

final_dataset = global_confirmed2.merge(global_deaths2[['Province/State','Country/Region','Date','Deaths']], 
                                      how = 'outer',left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])
final_df = final_dataset.merge(global_recovered2[['Province/State','Country/Region','Date','Recovered']], 
                                      how = 'outer',left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

final_df.isnull().sum()

Province/State       0
Country/Region       0
Lat                 91
Long                91
Date                 0
Confirmed           91
Deaths              91
Recovered         1365
dtype: int64

In [16]:
# Adding Month and Year as a new Column
final_df['Month-Year'] = final_df['Date'].dt.strftime('%b-%Y')
final_df

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year
0,Afghanistan,Afghanistan,33.0000,65.0000,2020-01-22,0.0,0.0,0.0,Jan-2020
1,Albania,Albania,41.1533,20.1683,2020-01-22,0.0,0.0,0.0,Jan-2020
2,Algeria,Algeria,28.0339,1.6596,2020-01-22,0.0,0.0,0.0,Jan-2020
3,Andorra,Andorra,42.5063,1.5218,2020-01-22,0.0,0.0,0.0,Jan-2020
4,Angola,Angola,-11.2027,17.8739,2020-01-22,0.0,0.0,0.0,Jan-2020
...,...,...,...,...,...,...,...,...,...
24110,Canada,Canada,NaN,NaN,2020-04-17,NaN,NaN,10545.0,Apr-2020
24111,Canada,Canada,NaN,NaN,2020-04-18,NaN,NaN,10964.0,Apr-2020
24112,Canada,Canada,NaN,NaN,2020-04-19,NaN,NaN,11847.0,Apr-2020
24113,Canada,Canada,NaN,NaN,2020-04-20,NaN,NaN,12543.0,Apr-2020


In [17]:
## Calculating Daily number #########
final_df2 = final_df.copy()

#creating a new date columns - 1
final_df2['Date - 1'] = final_df2['Date'] + pd.Timedelta(days=1)
final_df2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
final_df3 = final_df.merge(final_df2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'left',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

#minus_onedf.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1'}, inplace=True)

final_df3.head()

# Additional Calculations
final_df3['Confirmed Daily'] = final_df3['Confirmed'] - final_df3['Confirmed - 1']
final_df3['Deaths Daily'] = final_df3['Deaths'] - final_df3['Deaths - 1']
final_df3['Recovered Daily'] = final_df3['Recovered'] - final_df3['Recovered - 1']

print(final_df3.shape)

(24115, 17)


In [18]:
final_df3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1,Confirmed Daily,Deaths Daily,Recovered Daily
0,Afghanistan,Afghanistan,33.0000,65.0000,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,Albania,Albania,41.1533,20.1683,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,Algeria,Algeria,28.0339,1.6596,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,Andorra,Andorra,42.5063,1.5218,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,Angola,Angola,-11.2027,17.8739,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


In [19]:
##Since the data for 22-1-20 is NaN for all regions we add data manually####

final_df3['Confirmed Daily'].loc[final_df3['Date'] == '2020-01-22'] = final_df3['Confirmed']
final_df3['Deaths Daily'].loc[final_df3['Date'] == '2020-01-22'] = final_df3['Deaths']
final_df3['Recovered Daily'].loc[final_df3['Date'] == '2020-01-22'] = final_df3['Recovered']

# deleting columns
del final_df3['Confirmed - 1']
del final_df3['Deaths - 1']
del final_df3['Recovered - 1']
del final_df3['Date - 1']
del final_df3['Date Minus 1']

C:\Users\DELL\AppData\Local\Temp\ipykernel_6252\2371012293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df3['Confirmed Daily'].loc[final_df3['Date'] == '2020-01-22'] = final_df3['Confirmed']
C:\Users\DELL\AppData\Local\Temp\ipykernel_6252\2371012293.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df3['Deaths Daily'].loc[final_df3['Date'] == '2020-01-22'] = final_df3['Deaths']
C:\Users\DELL\AppData\Local\Temp\ipykernel_6252\2371012293.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

In [20]:
final_df3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily
0,Afghanistan,Afghanistan,33.0000,65.0000,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
1,Albania,Albania,41.1533,20.1683,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
2,Algeria,Algeria,28.0339,1.6596,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
3,Andorra,Andorra,42.5063,1.5218,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
4,Angola,Angola,-11.2027,17.8739,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0


In [21]:
# Exporting the data

final_df3.to_csv('CoronaVirus PowerBI Raw', sep='\t')